#**Application de la méthode par itération des valeurs sur le projet Magasin**

##**1. Téléchargement du programme Python**

In [ ]:
!git clone "https://github.com/AlexandreBourrieau/ApplicationRL.git"

In [ ]:
import os
import sys

sys.path.insert(1, "/content/ApplicationRL")

In [ ]:
!pip -q install meshio

##**2. Mise en place de l'environnement de travail**

In [ ]:
from magasin import Magasin
from utils import bgr8_to_jpeg
import ipywidgets
import numpy as np

from IPython.display import display

#####**2.1 Instanciation de la classe Magasin**

In [ ]:
Magasin = Magasin("ApplicationRL/maillage_carte.msh","ApplicationRL/image_magasin.png")

In [ ]:
Magasin.RECOMPENSE = 1
Magasin.RECOMPENSE_MUR = -10
Magasin.RECOMPENSE_NON_CIBLE = 0
Magasin.PROBA_MAX = 0.98

Magasin.RAZ()

#####**2.2 Saisie de l'objectif**

In [ ]:
def AfficheSelectionImage(change):
    numero = change['new']
    Magasin.ETAT_CIBLE = numero
    widget_image.value = bgr8_to_jpeg(Magasin.AfficheObjectifSurImage())

In [ ]:
# Création des widgets
widget_image = ipywidgets.Image(format='png',value=bgr8_to_jpeg(Magasin.image_magasin),width=640)
slider_numero = ipywidgets.IntSlider(value=80,min=0,max=Magasin.nombre_etats-1)

# Création du lien avec le slider
slider_numero.observe(AfficheSelectionImage, names='value')

# Création de l'interface d'acquisition
widget_presentation = ipywidgets.VBox([widget_image, slider_numero])

display(widget_presentation)

##**3. Algorithme par itération des valeurs**

On commence par initialiser l'environnement :

In [ ]:
Magasin.RECOMPENSE = 50
Magasin.RECOMPENSE_MUR = -0.1
Magasin.RECOMPENSE_NON_CIBLE = -0.1
Magasin.PROBA_MAX = 0.9

Magasin.RAZ()

L'algorithme par itération des valeurs à utilisé est donné ci-dessous: 

<center><img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/RL/Concept_RL31.png?raw=true" width="700"></center>

In [ ]:
# Algorithme d'itération des valeurs 
gamma = 0.9     # Facteur de remise
theta = 1e-10   # Seuil de précision

# Initialisation de la fonction des valeurs d'états à 0
Magasin.V_table = np.zeros(Magasin.nombre_etats, dtype=np.float32)
V_ = np.zeros(Magasin.nombre_etats, dtype=np.float32)


# Répétition de l'algorithme jusqu'à convergence
delta = 2*theta

while delta > theta:
  # Pour chaque état de l'environnement
  for etat in range(Magasin.nombre_etats):
    # Initialise la fonction de valeurs d'états pour chaque action à 0
    V = np.zeros((3), dtype=np.float32)

    # Calcule la nouvelle valeur de l'état en cours à l'aide
    # de l'équation optimale des valeurs d'états de Bellman
    for action in range(3):
      # Calcul la valeur de l'état en cours pour l'action courante
      # ainsi que les valeurs d'actions
      for etat_suivant in range(3):
        proba = Magasin.table_transitions[etat,action,etat_suivant]['proba_transition']
        recompense = Magasin.table_transitions[etat,action,etat_suivant]['recompense']
        index_etat_suivant = Magasin.table_transitions[etat,action,etat_suivant]['index_etat_suivant']

        # Itération de la valeur de l'état courant pour l'action en cours
        V[action] += proba*(recompense + gamma*Magasin.V_table[index_etat_suivant])

        # Itération de la valeur des actions 
        Magasin.Q_table[etat,action] +=  proba*(recompense + gamma*Magasin.V_table[index_etat_suivant])

    # Enregistre la valeur d'état maximale parmi toutes les actions
    Magasin.V_table[etat] = np.amax(V)
    
  # Compare la fonction des valeurs des états obtenue avec la précédente
  delta = np.max(np.abs(V_ - Magasin.V_table))
  print(delta)

  # Sauvegarde la fonction des valeurs des états pour la prochaine itération
  V_ = Magasin.V_table.copy()

In [ ]:
Magasin.Getdf_Vtable()

#####**Affichage de la carte des valeurs d'états**

In [ ]:
# Création de la colorMap
Magasin.CreationColorMap()

# Création des widgets
widget_image = ipywidgets.Image(format='png',value=bgr8_to_jpeg(Magasin.image_Vtable),width=640)

display(widget_image)

#####**Affichage du parcours**

In [ ]:
Magasin.Getdf_Qtable()

In [ ]:
# Création des widgets
widget_image = ipywidgets.Image(format='png',value=bgr8_to_jpeg(Magasin.image_magasin_objectif),width=640)
widget_iteration = ipywidgets.Text(value="0",description="Itération: ")
widget_etat = ipywidgets.Text(value="0",description="État: ")

widget_presentation = ipywidgets.VBox([widget_iteration,widget_etat,widget_image])

In [ ]:
import time

objectif_atteint = False
max_iteration = 500
freq_mise_a_jour = 1

Magasin.InitImageTrajectoire()

iteration = 0
etat_courant = Magasin._ETAT_DEPART

display(widget_presentation)

time.sleep(1)

while objectif_atteint is False and iteration <= max_iteration:
    widget_etat.value = str(etat_courant)

    # Récupère la table des valeurs d'action de l'état en cours
    Q_ = Magasin.Q_table[etat_courant,:]

    # Récupère l'action optimale
    # si plusieurs actions sont optimales, alors tire une au hasard
    #action = np.random.choice(np.where(Q_ == np.max(Q_))[0])
    action = np.argmax(Q_)

    # Simule l'action
    etat_courant, image_ = Magasin.SimuleAction(etat_courant,action)

    if etat_courant == Magasin.ETAT_CIBLE:
        objectif_atteint = True

    # Affiche la trajectoire
    if iteration%freq_mise_a_jour == 0:
      widget_image.value = bgr8_to_jpeg(image_)
      widget_iteration.value = str(iteration+1)

    iteration = iteration + 1
    time.sleep(0.7)
